In [16]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

SAVE_PATH = './regensburg_pediatric_appendicitis.csv'  

try:
    data = pd.read_csv(SAVE_PATH)
except:
    from ucimlrepo import fetch_ucirepo 
    regensburg_pediatric_appendicitis = fetch_ucirepo(id=938) 
    data = regensburg_pediatric_appendicitis['data']['features']
    data.to_csv(SAVE_PATH, index=False)
